In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx
import matplotlib
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go

import math
import os
import gc
import argparse
import torch
import optuna
import joblib
import pickle
import tifffile
import nibabel
import scipy.io
import pygsp
import scipy.ndimage
import pyod
import warnings
import hashlib
import sqlite3

from scipy.stats import entropy, kurtosis, skew
from sklearn.mixture import GaussianMixture
from torch_geometric.utils import dense_to_sparse
from sklearn.cluster import KMeans, BisectingKMeans, SpectralClustering
from sklearn.metrics import pairwise_distances

from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.cof import COF
from pyod.models.cblof import CBLOF
from pyod.models.kpca import KPCA

from torch import nn, Tensor
from torch.nn import Linear, Conv1d, LayerNorm, DataParallel, ReLU, Sequential, Parameter
from torch_geometric.nn.dense import mincut_pool, dense_mincut_pool
from torch_geometric.datasets import AttributedGraphDataset
from torch_geometric.utils import to_networkx, subgraph, to_dense_adj

import source.nn.models as models
import source.utils.utils as utils
import source.utils.fault_detection as fd

from source.utils.utils import roc_params, compute_auc, get_auc, best_mcc, best_f1score, otsuThresholding
from source.utils.utils import synthetic_timeseries, synthetic_binary
from source.utils.utils import plotly_signal


from importlib import reload
models = reload(models)
utils = reload(utils)

from pyprojroot import here
root_dir = str(here())

data_dir = os.path.expanduser('~/data/interim/')

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'DejaVu Serif'})

pd.set_option('display.max_rows', 500)


In [45]:
epochs_list = [1,5,10,25,50,100,150,200,300,500,750,1000,1250]


def skewscore(scores, skewth):
    sk = skew(scores)
    return (1-scores) if (sk<skewth) else scores

def train_model(model, X, G, device, weight_loss, lr):

    rng_seed = 0
    torch.manual_seed(rng_seed)
    torch.cuda.manual_seed(rng_seed)
    np.random.seed(rng_seed)

    loss_evo = []
    loss_mc_evo = []
    loss_o_evo = []
    S_partials = []

    # Node coordinates
    A = torch.tensor(G.W.toarray()).float() #Using W as a float() tensor
    A = A.to(device)    

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    model.reset_parameters()
    for epoch in range(1, 1+np.max(epochs_list)):

        optimizer.zero_grad()
        S, loss_mc, loss_o = model(X, A)
        loss = loss_mc + weight_loss*loss_o
        loss.backward()
        optimizer.step()
        loss_evo.append(loss.item())
        loss_mc_evo.append(loss_mc.item())
        loss_o_evo.append(loss_o.item())
        if epoch in epochs_list:
            S_partials.append(S)

    return S_partials, loss_mc_evo, loss_o_evo

def test_locality(model, datasets, epochs, weight_loss, lr, skewth, device):

    id_list = []
    sample_list = []
    auc_list = []
    f1_list = []
    mcc_list = []
    eig_list = []
    deg_list = []

    it = 0
    for dataset in datasets:

        print(f'Testing dataset {it}', flush=True)
        it+=1

        G = dataset['G']
        data = dataset['data']
        labels = dataset['labels']

        id = dataset['metadata']['id']
        n_samples = dataset['metadata']['samples']

        G.compute_fourier_basis()
        

        for sample in range(n_samples):
            X = torch.tensor(data[sample]).float().to(device)
            label = labels[sample]

            S_epoch = train_model(model, X, G, device, weight_loss, lr)[0]

            epochs_id = epochs_list.index(epochs)
            scores = S_epoch[epochs_id].detach().cpu().softmax(dim=1).max(dim=1)[0].numpy()

            scores = skewscore(scores, skewth)

            id_list.append(id)
            sample_list.append(sample)
            eig_list.append(G.e[1].round(3))
            deg_list.append(G.A.toarray().sum(axis=1).mean().round(3))
            auc_list.append(get_auc(scores, label).round(3))
            f1_list.append(best_f1score(scores, label).round(3))
            mcc_list.append(best_mcc(scores, label).round(3))

    
    df_test = pd.DataFrame({'id':id_list,
                            'sample_id':sample_list,
                            'Eig':eig_list,
                            'Deg':deg_list,
                            'AUC':auc_list,
                            'F1 score':f1_list,
                            'MCC':mcc_list})

    return df_test


In [8]:
G = pygsp.graphs.Sensor(200, Nc=150, n_try=1000, distribute=True, seed=0)
# G.plot()

px.imshow(G.A.toarray()).show()

171.1

In [28]:
datasets = []
Nc_list = [2, 5, 10, 15, 20, 30, 50, 75, 100, 150]

for nc in Nc_list:

    metadata = {'samples':50,
                'SEED':0,
                'GRAPH_SEED':0,
                'id':nc,
                'N':200,
                'T':50,
                'num_flips':16,
                'noise':0.1,
                'missing_ratio':0.1
                }

    G = pygsp.graphs.Sensor(metadata['N'], Nc=nc, n_try=1000, distribute=True, seed=metadata['GRAPH_SEED'])

    DATA_SEED = metadata['SEED']
    torch.manual_seed(DATA_SEED)
    torch.cuda.manual_seed(DATA_SEED)
    np.random.seed(DATA_SEED)

    dataset = {'data':[], 'labels':[], 'G':G, 'metadata':metadata}

    for sample in range(metadata['samples']):

        X, label = synthetic_binary(G,
                                    metadata['T'],
                                    metadata['num_flips'],
                                    metadata['noise'],
                                    metadata['missing_ratio']
                                  )
        dataset['data'].append(X)
        dataset['labels'].append(label)

    datasets.append(dataset)

In [29]:
device='cuda'
study = joblib.load(root_dir+f'/outputs/HP_training/SB_MC.pkl')
best_params = study.best_params
model = models.ClusterTS(metadata['T'], n_clusters=best_params['n_clusters'])
model = model.to(device)

In [46]:
df_test = test_locality(model, datasets,
                        epochs=best_params['N_epochs'],
                        weight_loss=best_params['weight_loss'],
                        lr=best_params['lr'],
                        skewth=best_params['skewth'],
                        device=device
                    )
df_test.to_parquet(root_dir+'/outputs/testing_mincut/df_loc_capacity_synthetic.parq')


Testing dataset 0
Testing dataset 1
Testing dataset 2
Testing dataset 3
Testing dataset 4
Testing dataset 5
Testing dataset 6
Testing dataset 7
Testing dataset 8
Testing dataset 9


In [52]:
df_test.groupby('id', as_index=False).mean()

,id,sample_id,Eig,Deg,AUC,F1 score,MCC
0,2,24.5,0.198,11.21,0.95898,0.92672,0.91062
1,5,24.5,0.190,11.21,0.96998,0.93090,0.91920
2,10,24.5,0.203,12.43,0.97344,0.96536,0.95614
3,15,24.5,0.311,16.81,0.91382,0.88226,0.85864
4,20,24.5,0.495,22.51,0.88792,0.81660,0.77668
5,30,24.5,0.850,34.01,0.85758,0.80794,0.75964
6,50,24.5,1.440,57.95,0.56500,0.33140,0.19926
7,75,24.5,1.841,88.75,0.56416,0.30920,0.17246
8,100,24.5,1.991,118.50,0.54138,0.29974,0.16168
9,150,24.5,2.056,171.10,0.53564,0.29262,0.15428


In [55]:
df = df_test.groupby('id', as_index=False).mean().melt(id_vars=['sample_id','AUC','F1 score', 'MCC'],
                  value_vars=['Eig','Deg'], var_name='Varname', value_name='Var')
df_long = df.melt(id_vars=['Var', 'Varname'], value_vars=['AUC', 'F1 score', 'MCC'],
                  var_name='Metric', value_name='Value')

fig = px.scatter(df_long, x='Var', y='Value', color='Metric', facet_col='Varname',
                 trendline='lowess', height=400, width=1000, template='plotly_white')

# Customize x-axis labels for each facet
fig.update_xaxes(matches=None)  # Allow each x-axis to be edited separately

# Set custom x-axis titles for each facet
fig.update_xaxes(title_text="Fiedler value", col=1, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black')  
fig.update_xaxes(title_text="Average degree", col=2, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.update_yaxes(range=[0.2,1], col=1, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.update_yaxes(range=[0.2,1], col=2, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.for_each_annotation(lambda a: a.update(text=""))

fig.update_layout(font =dict(family="Times New Roman", size=22),
                  plot_bgcolor = 'rgba(0, 0, 0, 0)',
                  paper_bgcolor = 'rgba(0, 0, 0, 0)',
                  legend=dict(x=0.65,
                              y=0.45,
                              xanchor="right",
                              yanchor="top",
                              bgcolor="rgba(255, 255, 255, 0.8)"
                            ),
                  margin=dict(l=20, r=20, t=1, b=40)
)

fig.write_image(root_dir+'/outputs/figs/TPAMI/capacity_synthetic.png')
fig.show()

In [82]:
datasets = []

device='cuda'

dataset = AttributedGraphDataset(root='../data/', name='BlogCatalog')
data = dataset[0]

# Select a specific class (for example, class 0)

n_features = 100

class_of_interest = [0]
node_mask = torch.tensor(np.isin(data.y, class_of_interest))

# Filter the nodes that belong to the selected class
selected_nodes = node_mask.nonzero(as_tuple=True)[0]

# Get the edges that only involve the selected nodes
sub_edge_index, _ = subgraph(selected_nodes, data.edge_index, relabel_nodes=True)

# Create a new data object for the subgraph (optional but clearer)
sub_data = data.clone()
sub_data.edge_index = sub_edge_index
sub_data.x = sub_data.x[selected_nodes, 0:n_features]
sub_data.y = sub_data.y[selected_nodes]

# Can simplify if using sparsify
G_orig = pygsp.graphs.Graph(to_dense_adj(sub_data.edge_index).squeeze())
G_orig.set_coordinates()
A_orig = torch.tensor(G_orig.W.toarray()).float() #Using W as a float() tensor
A_orig = A_orig.to(device)

Nc_list = [2, 5, 10, 15, 20, 30, 50, 75, 100, 150]

for nc in Nc_list:

    metadata = {'samples':50,
                'SEED':0,
                'GRAPH_SEED':0,
                'id':nc,
                'N':G_orig.N,
                'T':n_features,

                'n_anomalies': 30
                }
    
    A = utils.sparsify_by_corr(sub_data.x.to(device), A_orig, nc)
    G = pygsp.graphs.Graph(W=A.cpu().numpy(), coords=G.coords)

    DATA_SEED = metadata['SEED']
    torch.manual_seed(DATA_SEED)
    torch.cuda.manual_seed(DATA_SEED)
    np.random.seed(DATA_SEED)

    dataset = {'data':[], 'labels':[], 'G':G, 'metadata':metadata}

    for sample in range(metadata['samples']):

        X, label = utils.network_anomaly(G, sub_data.x, metadata['n_anomalies'])

        dataset['data'].append(X)
        dataset['labels'].append(label)

    datasets.append(dataset)

In [85]:
device='cuda'
study = joblib.load(root_dir+f'/outputs/HP_training/SB_MC.pkl')
best_params = study.best_params
model = models.ClusterTS(metadata['T'], n_clusters=best_params['n_clusters'])
model = model.to(device)

In [86]:
df_test = test_locality(model, datasets,
                        epochs=best_params['N_epochs'],
                        weight_loss=best_params['weight_loss'],
                        lr=best_params['lr'],
                        skewth=best_params['skewth'],
                        device=device
                    )
df_test.to_parquet(root_dir+'/outputs/testing_mincut/df_loc_capacity_BlogCatalog.parq')


Testing dataset 0
Testing dataset 1
Testing dataset 2
Testing dataset 3
Testing dataset 4
Testing dataset 5
Testing dataset 6
Testing dataset 7
Testing dataset 8
Testing dataset 9


In [2]:
df_test = pd.read_parquet(root_dir+'/outputs/testing_mincut/df_loc_capacity_BlogCatalog.parq')

In [4]:
df = df_test.groupby('id', as_index=False).mean().melt(id_vars=['sample_id','AUC','F1 score', 'MCC'],
                  value_vars=['Eig','Deg'], var_name='Varname', value_name='Var')
df_long = df.melt(id_vars=['Var', 'Varname'], value_vars=['AUC', 'F1 score', 'MCC'],
                  var_name='Metric', value_name='Value')

fig = px.scatter(df_long, x='Var', y='Value', color='Metric', facet_col='Varname',
                 trendline='lowess', height=400, width=1000, template='plotly_white')

# Customize x-axis labels for each facet
fig.update_xaxes(matches=None)  # Allow each x-axis to be edited separately

# Set custom x-axis titles for each facet
fig.update_xaxes(title_text="Fiedler value", col=1, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black')  
fig.update_xaxes(title_text="Average degree", col=2, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.update_yaxes(range=[0,1], col=1, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.update_yaxes(range=[0,1], col=2, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.for_each_annotation(lambda a: a.update(text=""))

fig.update_layout(font =dict(family="Times New Roman", size=22),
                  plot_bgcolor = 'rgba(0, 0, 0, 0)',
                  paper_bgcolor = 'rgba(0, 0, 0, 0)',
                  legend=dict(x=0.65,
                              y=0.45,
                              xanchor="right",
                              yanchor="top",
                              bgcolor="rgba(255, 255, 255, 0.8)"
                            ),
                  margin=dict(l=20, r=20, t=1, b=40)
)

fig.write_image(root_dir+'/outputs/figs/TPAMI/capacity_blogcatalog.png')
fig.show()

In [94]:
df_test_VR = pd.read_parquet(root_dir+'/outputs/testing_mincut/df_locality_VR.parq').drop(['DR'],axis=1)
df_test_DR = pd.read_parquet(root_dir+'/outputs/testing_mincut/df_locality_DR.parq').drop(['VR'],axis=1)
df_means_VR = df_test_VR.groupby('bin', as_index=False).mean().rename({'VR':'Var'}, axis=1)
df_means_VR['Varname'] = 'VR'
df_means_DR = df_test_DR.groupby('bin', as_index=False).mean().rename({'DR':'Var'}, axis=1)
df_means_DR['Varname'] = 'DR'
df = pd.concat([df_means_VR, df_means_DR])
df_long = df.melt(id_vars=['Var', 'Varname'], value_vars=['AUC', 'F1 score', 'MCC'],
                  var_name='Metric', value_name='Value')

fig = px.scatter(df_long, x='Var', y='Value', color='Metric', facet_col='Varname',
                 trendline='lowess', height=600, width=1000, template='plotly_white')

# Customize x-axis labels for each facet
fig.update_xaxes(matches=None)  # Allow each x-axis to be edited separately

# Set custom x-axis titles for each facet
fig.update_xaxes(title_text="VR", col=1, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black')  
fig.update_xaxes(title_text="DR", col=2, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.update_yaxes(range=[0.2,1], col=1, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.update_yaxes(range=[0.2,1], col=2, row=1, showline=True, mirror='allticks', linewidth=1, linecolor='black') 
fig.for_each_annotation(lambda a: a.update(text=""))

fig.update_layout(font =dict(family="Times New Roman", size=22),
                  plot_bgcolor = 'rgba(0, 0, 0, 0)',
                  paper_bgcolor = 'rgba(0, 0, 0, 0)',
                  legend=dict(x=0.98,
                              y=0.30,
                              xanchor="right",
                              yanchor="top",
                              bgcolor="rgba(255, 255, 255, 1)"
                            ),
                  margin=dict(l=20, r=20, t=1, b=40)
)

fig.write_image(root_dir+'/outputs/figs/TPAMI/locality_synthetic.png')
fig.show()


### Oil Spill

In [53]:
S_partials_list = []
label_list = []
shape_list = []

datasets = []

for im in range(1,19):

    mat = scipy.io.loadmat(data_dir+f'HSIoil/GM{im:02}.mat')
    downsample_factor = 0.1
    map = scipy.ndimage.zoom(mat['map'], zoom=downsample_factor, order=3)
    data_orig = scipy.ndimage.zoom(mat['img'], zoom=(downsample_factor, downsample_factor, 1), order=3)
    X = data_orig.reshape(data_orig.shape[0]*data_orig.shape[1],data_orig.shape[2])
    # X = torch.tensor(X.astype(np.float32)).float()

    metadata = {'samples':1,
                'id':im,
                'N':X.shape[0],
                'T':X.shape[1],
                'downsample':downsample_factor
                }

    print(im)

    G = pygsp.graphs.Grid2d(data_orig.shape[0],data_orig.shape[1])

    dataset = {'data':[X], 'labels':[map.reshape(-1,)], 'G':G, 'metadata':metadata}
    datasets.append(dataset)

    # coords = G.coords
    # A = G.W.toarray()
    # idx = np.lexsort((-coords[:, 1], coords[:, 0]))
    # A = torch.tensor(A[np.ix_(idx,idx)]).float()
    # A = A.to(device)

    # n_timestamps = X.shape[1]
    # n_clusters = 5
    # n_extra_feats = 0
    # weight_loss = 1

    # model = models.ClusterTS(n_timestamps, n_clusters, n_extra_feats)
    # model = model.to(device)

    # epochs_list = [1,25,50,100,200,500,1000]
    # S_partials, lmc, lo = train_cluster(epochs_list, model, X, G, device, weight_loss)
    # S_partials_list.append(S_partials)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [48]:
device='cuda:2'
study = joblib.load(root_dir+f'/outputs/HP_training/SB_MC.pkl')
best_params = study.best_params
model = models.ClusterTS(metadata['T'], n_clusters=best_params['n_clusters'])
model = model.to(device)

In [52]:
df_test = test_locality(model, datasets,
                        epochs=best_params['N_epochs'],
                        weight_loss=best_params['weight_loss'],
                        lr=best_params['lr'],
                        skewth=best_params['skewth'],
                        device=device
                    )

Testing dataset 0
Testing dataset 1
Testing dataset 2
Testing dataset 3
Testing dataset 4
Testing dataset 5
Testing dataset 6
Testing dataset 7
Testing dataset 8


In [50]:
df_test

,id,VR,DR,sample_id,AUC,F1 score,MCC
0,1,0.930545,0.775748,0,0.298,0.054,0.004
1,2,0.473281,0.670084,0,0.039,0.017,-0.002
2,3,0.518550,0.801663,0,0.975,0.761,0.752
3,4,0.519887,0.704821,0,0.050,0.052,-0.004


In [ ]:
datasets = []
for bin in DR_bins:

    metadata = {'samples':50,
                'bin':bin,
                'GRAPH_SEED':0,
                'N':400,
                'T':100,
                'wave_params': {'wave_speed': 0.05, 'frequency': 0.25, 'w_amp':0.5},
                'noise':0.1
                }

    G = pygsp.graphs.Sensor(metadata['N'], n_try=1000, distribute=True, seed=metadata['GRAPH_SEED'])

    dataset = {'data':[], 'labels':[], 'G':G, 'metadata':metadata}

    for seed in DR_bins[bin]:
        np.random.seed(seed)
        label = np.zeros(metadata['N'])
        label[select_walk(G, 40, 40)] = 1

        X = locality_data(G.coords, metadata['T'], label, metadata['wave_params'], noise=0.2)[0]
        dataset['data'].append(X)
        dataset['labels'].append(label)

    datasets.append(dataset)

In [ ]:
df_test = test_locality(model, datasets,
                        epochs=best_params['N_epochs'],
                        weight_loss=best_params['weight_loss'],
                        lr=best_params['lr'],
                        skewth=best_params['skewth'],
                        device=device
                    )

In [13]:
px.imshow(map).show()